In [1]:
import psycopg2
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)


# Tell MatPlotLib to draw plots inline with the code outputs
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

conn = psycopg2.connect( 
                        host = 'project.cgxhdwn5zb5t.us-east-1.rds.amazonaws.com',
                        port = 5432, 
                        user = 'postgres',
                        password = 'Admin123',
                        database = 'postgres')
cursor = conn.cursor()

OperationalError: could not connect to server: Operation timed out
	Is the server running on host "project.cgxhdwn5zb5t.us-east-1.rds.amazonaws.com" (34.193.96.39) and accepting
	TCP/IP connections on port 5432?


In [ ]:
DEC2FLOAT = psycopg2.extensions.new_type(
    psycopg2.extensions.DECIMAL.values,
    'DEC2FLOAT',
    lambda value, curs: float(value) if value is not None else None)
psycopg2.extensions.register_type(DEC2FLOAT)

In [ ]:
cursor.execute('SELECT * FROM household_181')
rows = cursor.fetchall()
col_names = []
for elt in cursor.description:
    col_names.append(elt[0])

df = pd.DataFrame(data=rows, columns=col_names )

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
# Use pandas to count the number of each unique value for each state.
neighbor_g = df.groupby('omb13cbsa')[['ratingnh']].count()
neighbor_g = neighbor_g.sort_values('ratingnh')
neighbor_g

In [ ]:
# need help?? metro = df_neighbor[(df_neighbor['omb13cbsa'] == 99999) or (df_neighbor['omb13cbsa'] == 99998)]

In [ ]:
#Unique Neighborhood Rating
df.ratingnh.unique()

In [ ]:
# Get names of indexes for which column ratingnh has value less than 0
indexNames = df[ df['ratingnh'] < 0 ].index
 
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

In [ ]:
#Average Neighborhood rating
df['ratingnh'].mean()

# Visualization


In [ ]:
fig = plt.figure(figsize=(7,10))
ax = fig.add_subplot(212)
ax.hist(df['ratingnh'], bins=50, range=(df['ratingnh'].min(),df['ratingnh'].max()))
plt.title('Rating Distribution with 50 Bins')
plt.xlabel('Rating')
plt.ylabel('Count of Rating')
plt.show()

In [ ]:
df.ratingnh.unique()

In [ ]:
#Nation-wide distribution of Satisfaction levels for NEIGHBORHOOD?
sns.relplot(x="ratingnh", y='control',data=df)

In [ ]:
#filter on 47900: Washington-Arlington-Alexandria, DC-VA-MD-WV|| states only
dmv = df[(df['omb13cbsa'] == 47900)]

In [ ]:
#DMV wide distribution of Satisfaction levels for NEIGHBORHOOD?
sns.relplot(x="ratingnh", y="control", data=dmv)

In [ ]:
df.omb13cbsa.unique()

In [ ]:
#Regression and Residual plot between total household income and neigborhood rating

#tips = sns.load_dataset("ratingnh")
sns.regplot(x="hincp", y="ratingnh", data=df, ci=None)
plt.title("Rating Regression")
plt.xlabel('Total Household Income', size=12)
plt.ylabel('Neighborhood Rating', size=12)
plt.show()

sns.residplot(x="hincp", y="ratingnh", data=df)
plt.title("Rating Residuals")
plt.xlabel('Total Household Income', size=12)
plt.ylabel('Neighborhood Rating', size=12)
plt.show()

In [ ]:
# Label the state names
LABEL_MAP = {
    12060: "GA",
    12580: "MD",
    13820: "AL",
    14460: "MA-NH",
    16980: "IL-IN-WI",
    17140: "OH-KY-IN",
    17460: "OH",
    19100: "TX",
    19740: "CO",
    19820: "MI",
    26420: "TX",
    28140: "MO-KS",
    29820: "NV",
    31080: "CA",
    32820: "TN-MS-AR",
    33100: "FL",
    33340: "WI",
    33460: "MN-WI",
    35380: "LA",
    35620: "NY-NJ-PA",
    36420: "OK",
    37980: "PA-NJ-DE-MD",
    38060: "AZ",
    38300: "PA",
    38900: "OR-WA",
    39580: "NC",
    40060: "VA",
    40140: "CA",
    40380: "NY",
    41700: "TX",
    41860: "San Francisco, CA",
    41940: "San Jose-Sunnyvale-Santa Clara, CA",
    42660: "WA",
    45300: "Tampa-St. Petersburg-Clearwater, FL",
    47900: "DC-VA-MD-WV",
    99998: "All other metropolitan areas",
    99999: "Not in a metropolitan area"
}

In [ ]:
## Convert state column labels into text
df["omb13cbsa"] = df["omb13cbsa"].map(LABEL_MAP)
df["omb13cbsa"]

In [ ]:
f, ax = plt.subplots(figsize=(20,20))
sns.boxplot(x='omb13cbsa', y='ratingnh', data=df, palette='vlag')
sns.swarmplot(x='omb13cbsa', y='ratingnh', data=df, size=2, color='0.3')
plt.title('Rating By State', size=14)
plt.xlabel('State', size=12)
plt.ylabel('Rating', size=12)
plt.show()